In [1]:
import os
import pandas as pd
from IPython.display import display

DF_1_PATH = './vep_benign_variants_with_cluster.csv'
DF_2_PATH = './vep_pathogenic_variants_with_cluster.csv'

DF_1 = pd.read_csv(DF_1_PATH, sep='\t')
DF_2 = pd.read_csv(DF_2_PATH, sep='\t')

display(DF_1)
display(DF_2)

,uniprot,position,WT,Mut,cluster,category
0,A0A075B6H7,39,V,A,D2HJ94,LB/B
1,A0A075B6H7,55,T,S,D2HJ94,LB/B
2,A0A075B6H7,78,S,G,D2HJ94,LB/B
3,A0A075B6H8,27,T,I,A0A0H4LVB5,LB/B
4,A0A075B6I3,61,L,P,G1PYU6,LB/B
...,...,...,...,...,...,...
58241,Q9Y6Z7,179,R,W,L9LCR2,LB/B
58242,W5XKT8,246,G,S,A0A2Y9FYN0,LB/B
58243,W5XKT8,265,W,R,A0A2Y9FYN0,LB/B
58244,W6CW81,40,L,Q,A0A2K6E921,LB/B


,uniprot,position,WT,Mut,cluster,category
0,A0A1B0GTW7,31,S,F,A0A286YEC0,LP/P
1,A0A1B0GTW7,384,S,L,A0A286YEC0,LP/P
2,A0AVF1,263,N,S,A0A1Y1HXI3,LP/P
3,A0PJY2,278,H,Y,A0A091DC32,LP/P
4,A0PK11,165,T,K,A0A093GHV1,LP/P
...,...,...,...,...,...,...
31836,Q9Y6X9,413,V,F,A0A226P8Q3,LP/P
31837,Q9Y6X9,431,A,V,A0A226P8Q3,LP/P
31838,Q9Y6Y1,955,R,W,K7DY74,LP/P
31839,Q9Y6Y1,1077,Y,C,K7DY74,LP/P


## STANDARDIZE HUMSAVAR COLUMNS

In [15]:
import re

AA_DICT_LTS = {'VAL':'V', 'ILE':'I', 'LEU':'L', 'GLU':'E', 'GLN':'Q',
'ASP':'D', 'ASN':'N', 'HIS':'H', 'TRP':'W', 'PHE':'F', 'TYR':'Y',
'ARG':'R', 'LYS':'K', 'SER':'S', 'THR':'T', 'MET':'M', 'ALA':'A',
'GLY':'G', 'PRO':'P', 'CYS':'C', 'SEC': 'U'}

cols_to_drop = ['gene_name','FTId', 'change', 'dbSNP', 'disease_name']

def format_mutation(df: pd.DataFrame):
   
   new_cols = {
      'position': [],
      'WT': [],
      'Mut': []
   }
   
   for row in df.itertuples():
      
      mutation: str = getattr(row, 'change').replace('p.','')
      position = re.search(r'(\d+)',mutation).group(1)
      AA = mutation.split(position)
      WT = AA_DICT_LTS[AA[0].upper()]
      Mut = AA_DICT_LTS[AA[1].upper()]
      
      new_cols['position'].append(position)
      new_cols['WT'].append(WT)
      new_cols['Mut'].append(Mut)
      
   formatted_df = df.assign(**new_cols)
   
   return formatted_df

HUMSAVAR_DF = format_mutation(HUMSAVAR_DF).drop(columns=cols_to_drop).rename(columns={'AC': 'uniprot'})

# remove uncertain significance
US_mask = HUMSAVAR_DF['category'] != 'US'

HUMSAVAR_DF = HUMSAVAR_DF[US_mask].reset_index().drop(columns=['index'])

display(HUMSAVAR_DF)

,uniprot,category,cluster,position,WT,Mut
0,P04217,LB/B,A0A1U7UUV9,52,H,R
1,P04217,LB/B,A0A1U7UUV9,395,H,R
2,Q9NQ94,LB/B,V9KAZ0,555,V,M
3,Q9NQ94,LB/B,V9KAZ0,558,A,S
4,P01023,LB/B,A0A091S656,704,R,H
...,...,...,...,...,...,...
72162,Q8N402,LB/B,G3QPU9,131,T,P
72163,Q96M66,LB/B,A0A0D9R751,37,R,H
72164,Q96M66,LB/B,A0A0D9R751,171,R,S
72165,Q9N2K0,LB/B,Q5G5C9,81,V,L


## COMBINE DATAFRAMES AND REMOVE DUPLICATES

In [2]:
COMBINED_DF = pd.concat([DF_1, DF_2]).reset_index().drop(columns=['index'])

print('COMBINED DF')
display(COMBINED_DF)

encountered_dict = {}

def generate_key(tup):
   return getattr(tup, 'uniprot') + str(getattr(tup, 'position')) + getattr(tup, 'WT') + getattr(tup, 'Mut')

boolMask = []
REDUNDANT_SET = []

for row in COMBINED_DF.itertuples():
   key = generate_key(row)
   try:
      if encountered_dict[key] == True:
         boolMask.append(False)
         REDUNDANT_SET.append(getattr(row, 'Index'))
   except KeyError:
      encountered_dict[key] = True
      boolMask.append(True)

NR_COMBINED_DF = COMBINED_DF[boolMask].reset_index().drop(columns=['index'])

REDUNDANT_ROWS = COMBINED_DF.iloc[REDUNDANT_SET]

print('NON-REDUNDANT COMBINED DF')
display(NR_COMBINED_DF)

print('REDUNDANT ROWS')
display(REDUNDANT_ROWS)

COMBINED DF


,uniprot,position,WT,Mut,cluster,category
0,A0A075B6H7,39,V,A,D2HJ94,LB/B
1,A0A075B6H7,55,T,S,D2HJ94,LB/B
2,A0A075B6H7,78,S,G,D2HJ94,LB/B
3,A0A075B6H8,27,T,I,A0A0H4LVB5,LB/B
4,A0A075B6I3,61,L,P,G1PYU6,LB/B
...,...,...,...,...,...,...
90082,Q9Y6X9,413,V,F,A0A226P8Q3,LP/P
90083,Q9Y6X9,431,A,V,A0A226P8Q3,LP/P
90084,Q9Y6Y1,955,R,W,K7DY74,LP/P
90085,Q9Y6Y1,1077,Y,C,K7DY74,LP/P


NON-REDUNDANT COMBINED DF


,uniprot,position,WT,Mut,cluster,category
0,A0A075B6H7,39,V,A,D2HJ94,LB/B
1,A0A075B6H7,55,T,S,D2HJ94,LB/B
2,A0A075B6H7,78,S,G,D2HJ94,LB/B
3,A0A075B6H8,27,T,I,A0A0H4LVB5,LB/B
4,A0A075B6I3,61,L,P,G1PYU6,LB/B
...,...,...,...,...,...,...
90078,Q9Y6X9,413,V,F,A0A226P8Q3,LP/P
90079,Q9Y6X9,431,A,V,A0A226P8Q3,LP/P
90080,Q9Y6Y1,955,R,W,K7DY74,LP/P
90081,Q9Y6Y1,1077,Y,C,K7DY74,LP/P


REDUNDANT ROWS


,uniprot,position,WT,Mut,cluster,category
64825,P04637,72,P,R,L5JZ91,LP/P
67297,P10721,541,M,L,Q2PP56,LP/P
77931,P69905,69,N,K,A0A1K0FU75,LP/P
81835,Q15831,354,F,L,F6ZGN9,LP/P


## OUTPUT TO CSV

In [3]:
COMBINED_DF.to_csv('./vep_variants_combined.csv', sep='\t', index=False)
NR_COMBINED_DF.to_csv('./vep_variants_combined_nr.csv', sep='\t', index=False)